# **Exámen del Primer Bimestre de Recuperación de la Información**
## Instrucciones:
En este examen, los estudiantes deberán diseñar e implementar un sistema básico de recuperación de información utilizando la base de datos Rotten Tomatoes movies and critic reviews disponible en Kaggle. El objetivo es responder consultas relacionadas con la temática de las películas y sus características.

## Requerimientos:


1.   Preprocesamiento de Datos
2.   Construcción del Sistema
3.   Simulación de Consultas: El sistema debe mostrar un listado de las películas más relevantes para cada consulta, ordenadas segpun su relevancia.
4.   Análisis de resultados





## **Parte 1: Preprocesamiento del Corpus**


1.1. Cargar el Corpus:


*   La base de datos de Reviews nos permitirá generar un corpus que se base en la descripción y calificación que los críticos han dado a las películas evaluadas en la plataforma.





In [10]:
import pandas as pd

# Leer el archivo como DataFrame
reviews_df = pd.read_csv("/content/rotten_tomatoes_critic_reviews.csv")

# Mostrar las primeras filas para entender la estructura
print(reviews_df.head())
print(reviews_df.info())

  rotten_tomatoes_link      critic_name  top_critic           publisher_name  \
0            m/0814255  Andrew L. Urban       False           Urban Cinefile   
1            m/0814255    Louise Keller       False           Urban Cinefile   
2            m/0814255              NaN       False      FILMINK (Australia)   
3            m/0814255     Ben McEachen       False  Sunday Mail (Australia)   
4            m/0814255      Ethan Alter        True       Hollywood Reporter   

  review_type review_score review_date  \
0       Fresh          NaN  2010-02-06   
1       Fresh          NaN  2010-02-06   
2       Fresh          NaN  2010-02-09   
3       Fresh        3.5/5  2010-02-09   
4      Rotten          NaN  2010-02-10   

                                      review_content  
0  A fantasy adventure that fuses Greek mythology...  
1  Uma Thurman as Medusa, the gorgon with a coiff...  
2  With a top-notch cast and dazzling special eff...  
3  Whether audiences will get behind The Light

1.2. Limpieza del texto:


*   Los atributos de interés en esta base de datos son las diversas críticas para las películas, por lo que el preprocesamiento se concentra en gran parte en la columna de "review_content"



In [11]:
import re

# Asegurarnos de que la columna 'Synopsis' no tiene valores nulos
reviews_df['review_content'] = reviews_df['review_content'].fillna('')

# Función para limpiar el texto
def limpiar_texto(texto):
    texto = re.sub(r'[^A-Za-z\s]', '', texto)  # Eliminar caracteres especiales y números
    texto = re.sub(r'\s+', ' ', texto)  # Reemplazar múltiples espacios por uno solo
    return texto.strip()

# Aplicar la limpieza al texto de la sinopsis
reviews_df['Cleaned_Review'] = reviews_df['review_content'].apply(limpiar_texto)

# Mostrar el resultado de la limpieza
print(reviews_df[['review_content', 'Cleaned_Review']].head())

                                      review_content  \
0  A fantasy adventure that fuses Greek mythology...   
1  Uma Thurman as Medusa, the gorgon with a coiff...   
2  With a top-notch cast and dazzling special eff...   
3  Whether audiences will get behind The Lightnin...   
4  What's really lacking in The Lightning Thief i...   

                                      Cleaned_Review  
0  A fantasy adventure that fuses Greek mythology...  
1  Uma Thurman as Medusa the gorgon with a coiffu...  
2  With a topnotch cast and dazzling special effe...  
3  Whether audiences will get behind The Lightnin...  
4  Whats really lacking in The Lightning Thief is...  


1.3. Texto a minúsculas

In [14]:
# Normalizar el texto a minúsculas
reviews_df['Normalized_Review'] = reviews_df['Cleaned_Review'].str.lower()

# Mostrar el resultado de la normalización
print(reviews_df[['Cleaned_Review', 'Normalized_Review']].head())

# Expandir contracciones
!pip install contractions
from contractions import fix

reviews_df['Normalized_Review'] = reviews_df['Normalized_Review'].apply(fix)

                                      Cleaned_Review  \
0  A fantasy adventure that fuses Greek mythology...   
1  Uma Thurman as Medusa the gorgon with a coiffu...   
2  With a topnotch cast and dazzling special effe...   
3  Whether audiences will get behind The Lightnin...   
4  Whats really lacking in The Lightning Thief is...   

                                   Normalized_Review  
0  a fantasy adventure that fuses greek mythology...  
1  uma thurman as medusa the gorgon with a coiffu...  
2  with a topnotch cast and dazzling special effe...  
3  whether audiences will get behind the lightnin...  
4  whats really lacking in the lightning thief is...  


1.4. Tokenización

In [15]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

# Función para tokenizar el texto
def tokenizar_texto(texto):
    return nltk.word_tokenize(texto)

# Aplicar la tokenización al texto normalizado
reviews_df['Tokens'] = reviews_df['Normalized_Review'].apply(tokenizar_texto)

# Mostrar el resultado de la tokenización
print(reviews_df[['Normalized_Review', 'Tokens']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                   Normalized_Review  \
0  a fantasy adventure that fuses greek mythology...   
1  uma thurman as medusa the gorgon with a coiffu...   
2  with a topnotch cast and dazzling special effe...   
3  whether audiences will get behind the lightnin...   
4  what is really lacking in the lightning thief ...   

                                              Tokens  
0  [a, fantasy, adventure, that, fuses, greek, my...  
1  [uma, thurman, as, medusa, the, gorgon, with, ...  
2  [with, a, topnotch, cast, and, dazzling, speci...  
3  [whether, audiences, will, get, behind, the, l...  
4  [what, is, really, lacking, in, the, lightning...  


1.5. Eliminación de StopWords

In [16]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# Obtener las stopwords en inglés
stop_words = set(stopwords.words('english'))

# Función para eliminar stopwords
def eliminar_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

# Aplicar la eliminación de stopwords
reviews_df['Filtered_Tokens'] = reviews_df['Tokens'].apply(eliminar_stopwords)

# Mostrar el resultado después de eliminar las stopwords
print(reviews_df[['Tokens', 'Filtered_Tokens']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                              Tokens  \
0  [a, fantasy, adventure, that, fuses, greek, my...   
1  [uma, thurman, as, medusa, the, gorgon, with, ...   
2  [with, a, topnotch, cast, and, dazzling, speci...   
3  [whether, audiences, will, get, behind, the, l...   
4  [what, is, really, lacking, in, the, lightning...   

                                     Filtered_Tokens  
0  [fantasy, adventure, fuses, greek, mythology, ...  
1  [uma, thurman, medusa, gorgon, coiffure, writh...  
2  [topnotch, cast, dazzling, special, effects, t...  
3  [whether, audiences, get, behind, lightning, t...  
4  [really, lacking, lightning, thief, genuine, s...  


1.6. Implementar Lematización:


*   Usar lematización en un contexto de críticas tiene varios beneficios, como obtener el contexto gramatical.



In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Función para lematizar palabras
def lematizar_palabras(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# Aplicar la lematización
reviews_df['Lemmatized_Tokens'] = reviews_df['Filtered_Tokens'].apply(lematizar_palabras)

# Mostrar el resultado después de la lematización
print(reviews_df[['Filtered_Tokens', 'Lemmatized_Tokens']].head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                     Filtered_Tokens  \
0  [fantasy, adventure, fuses, greek, mythology, ...   
1  [uma, thurman, medusa, gorgon, coiffure, writh...   
2  [topnotch, cast, dazzling, special, effects, t...   
3  [whether, audiences, get, behind, lightning, t...   
4  [really, lacking, lightning, thief, genuine, s...   

                                   Lemmatized_Tokens  
0  [fantasy, adventure, fuse, greek, mythology, c...  
1  [uma, thurman, medusa, gorgon, coiffure, writh...  
2  [topnotch, cast, dazzling, special, effect, ti...  
3  [whether, audience, get, behind, lightning, th...  
4  [really, lacking, lightning, thief, genuine, s...  


## **Parte 2: Construcción del Sistema**


2.1. Vectorización:


*   En el inicializador del vectorizador TF-IDF se incluyen umbrales definidos manualmente:


*   max_df=0.8: Esto excluye palabras demasiado comunes que podrían no aportar significado, como palabras específicas del dominio que aparecen en casi todos los documentos.
*   min_df=2: Garantiza que solo se incluyan palabras que aparezcan al menos en 2 documentos, reduciendo términos raros que podrían añadir ruido.
*   stop_words='english': Capa adicional de seguridad contra palabras irrelevantes.





In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combinar los tokens lematizados en un texto por película
reviews_df['Processed_Text'] = reviews_df['Lemmatized_Tokens'].apply(lambda tokens: ' '.join(tokens))

# Inicializar el vectorizador TF-IDF con n-grams
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.8,           # Ignorar términos que aparezcan en más del 80% de los documentos
    min_df=2,             # Ignorar términos que aparezcan en menos de 2 documentos
    stop_words='english', # Remover stopwords adicionales (si aplica)
    ngram_range=(1, 2)    # Incluir unigramas y bigramas
)

# Construir la matriz TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(reviews_df['Processed_Text'])

# Obtener los términos (palabras clave) y la matriz resultante
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"Matriz TF-IDF construida con {tfidf_matrix.shape[0]} documentos y {tfidf_matrix.shape[1]} términos únicos.")

# Inspeccionar los primeros 10 términos y sus puntajes en el primer documento
document_idx = 0  # Índice del documento a inspeccionar
feature_array = tfidf_matrix[document_idx].toarray()[0]
top_features_idx = feature_array.argsort()[-10:][::-1]  # Índices de los 10 términos más relevantes
top_features = [(feature_names[i], feature_array[i]) for i in top_features_idx]
print(f"Términos destacados para el documento {document_idx}: {top_features}")

Matriz TF-IDF construida con 1130017 documentos y 1480534 términos únicos.
Términos destacados para el documento 0: [('mythology contemporary', 0.30492540684370734), ('american place', 0.29858674528208495), ('value couple', 0.29858674528208495), ('place value', 0.28331424202000316), ('thrill visual', 0.28331424202000316), ('greek mythology', 0.2420717676156381), ('contemporary american', 0.22974143693485558), ('couple year', 0.2231021763764049), ('visual spectacle', 0.2187295947746524), ('fantasy adventure', 0.212281039894362)]


Observaciones generales de los resultados:
* Términos como mythology contemporary, greek mythology, y fantasy adventure
destacan debido a su especificidad. Estos términos probablemente no aparecen de forma recurrente en otros documentos, lo que los hace relevantes.
* Bigrams como american place y visual spectacle probablemente aportan contexto o describen características específicas del contenido del documento.
* Los términos value couple, place value, y couple year podrían referirse a expresiones particulares de este documento que no son tan comunes en otros, aumentando su peso en el puntaje.

## **Parte 3: Simulación de Consultas**


3.1. Carga del CSV correspondiente a películas:


*   Este paso es necesario ya que el CSV correspondiente a las Reviews carece del título de las películas, por lo que es clave asociarlos para los siguientes pasos.



In [22]:
movies_df = pd.read_csv('/content/rotten_tomatoes_movies.csv')  # Ruta al archivo de movies

3.2. Eliminación de atributos:


*   En el DataFrame de Reviews se eliminarán las columnas que no fueron preprocesadas de ninguna manera y dejando únicamente las columnas de ID y a de los pasos del preprocesamiento para review_content.

*   En el DataFrame de Movies se eliminarán las columnas que no son relevantes para este caso y se dejarán únicamente las columnas de ID y de movie_title



In [23]:
# Eliminar columnas innecesarias en reviews_df
reviews_columns_to_drop = [
    'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date'
]

reviews_df.drop(columns=reviews_columns_to_drop, inplace=True)

# Eliminar columnas innecesarias en movies_df
movies_columns_to_drop = [
    'movie_info', 'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
    'actors', 'original_release_date', 'streaming_release_date', 'runtime', 'production_company',
    'tomatometer_status', 'tomatometer_rating', 'tomatometer_count', 'audience_status',
    'audience_rating', 'audience_count', 'tomatometer_top_critics_count',
    'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'
]

movies_df.drop(columns=movies_columns_to_drop, inplace=True)

# Verificar el resultado
print("Columnas de reviews_df después de la eliminación:")
print(reviews_df.columns)

print("\nColumnas de movies_df después de la eliminación:")
print(movies_df.columns)


Columnas de reviews_df después de la eliminación:
Index(['rotten_tomatoes_link', 'review_content', 'Cleaned_Review',
       'Normalized_Review', 'Tokens', 'Filtered_Tokens', 'Lemmatized_Tokens',
       'Processed_Text'],
      dtype='object')

Columnas de movies_df después de la eliminación:
Index(['rotten_tomatoes_link', 'movie_title'], dtype='object')


3.3. Preprocesamiento de las Consultas:

*   Esto asegura que la consulta esté en el mismo formato que los documentos de las reseñas, lo cual es clave para obtener resultados coherentes.





In [27]:
# Función para preprocesar la consulta
def preprocesar_consulta(consulta, stop_words):
    # Tokenizar, filtrar stopwords y lematizar
    tokens = nltk.word_tokenize(consulta.lower())  # Tokenización simple
    tokens_filtrados = eliminar_stopwords(tokens)
    tokens_lemmatizados = lematizar_palabras(tokens_filtrados)
    return ' '.join(tokens_lemmatizados)

3.4. Cálculo de la Similitud Coseno (Entre las consultas y la matriz TF-IDF):
* Este cálculo es fundamental para determinar qué tan relevantes son las películas para la consulta.
* En este cálculo se toma la consulta procesada y se realiza el cálculo de la similitud coseno con la matriz TF-IDF
* Ordenamos los índices de similitud de mayor a menor para mejores resultados
* Tomamos los 5 índices más relevantes (solo si es que los umbrales definidos en el preprocesamiento de la matriz TF-IDF lo permiten)
* Comparar entre los ID del DataFrame de Movies y los de Review para obtener el título de la película
* Excluir resultados repetidos (esto puede suceder ya que cada película tiene varias reviews)

In [32]:
import time
from sklearn.metrics.pairwise import cosine_similarity

# Función para simular consulta y eliminar duplicados en los títulos de películas
def simular_consulta(consulta, tfidf_vectorizer, tfidf_matrix, reviews_df, movies_df):
    # Iniciar el temporizador
    start_time = time.time()

    # Preprocesar la consulta
    consulta_procesada = preprocesar_consulta(consulta, stop_words)

    # Convertir la consulta procesada a la misma forma que los documentos
    consulta_tfidf = tfidf_vectorizer.transform([consulta_procesada])

    # Calcular la similitud coseno entre la consulta y los documentos
    similitudes = cosine_similarity(consulta_tfidf, tfidf_matrix).flatten()

    # Ordenar los índices por similitud (de mayor a menor)
    indices_ordenados = similitudes.argsort()[::-1]

    # Obtener los 5 índices más relevantes (si es posible)
    top_indices = indices_ordenados[:5]

    # Obtener los IDs de las películas más relevantes desde el DataFrame de Reviews
    peliculas_relevantes_reviews = reviews_df.iloc[top_indices]['rotten_tomatoes_link'].values

    # Obtener los títulos de las películas más relevantes desde el DataFrame de Movies
    peliculas_relevantes_movies = movies_df[movies_df['rotten_tomatoes_link'].isin(peliculas_relevantes_reviews)]

    # Resultados a mostrar (ID de Reviews y Títulos de Movies con la similitud)
    resultados = []
    vistos = set()  # Conjunto para rastrear IDs de películas ya procesados

    for idx in top_indices:
        id_review = reviews_df.iloc[idx]['rotten_tomatoes_link']
        id_movie = peliculas_relevantes_movies[peliculas_relevantes_movies['rotten_tomatoes_link'] == id_review]['movie_title'].values[0]
        similitud = similitudes[idx]

        # Evitar duplicados
        if id_movie not in vistos:
            resultados.append((id_review, id_movie, similitud))
            vistos.add(id_movie)  # Añadir al conjunto de vistas para no procesarlo de nuevo

    # Medir el tiempo de ejecución
    execution_time = time.time() - start_time

    # Mostrar los resultados
    print(f"Consulta: {consulta}")
    print(f"Tiempo de procesamiento: {execution_time:.4f} segundos")
    print("Resultados más relevantes:")
    for id_review, id_movie, similitud in resultados:
        print(f"ID de Review: {id_review} | Título de la Película: {id_movie} | Similitud Coseno: {similitud:.4f}")

    return resultados

3.5. Ejemplos de consultas:

In [33]:
# Ejemplo de consulta
consulta = "Movies about time travel"
resultados = simular_consulta(consulta, tfidf_vectorizer, tfidf_matrix, reviews_df, movies_df)

Consulta: Movies about time travel
Tiempo de procesamiento: 1.0260 segundos
Resultados más relevantes:
ID de Review: m/time_travelers_wife | Título de la Película: The Time Traveler's Wife | Similitud Coseno: 0.6413
ID de Review: m/butterfly_effect | Título de la Película: The Butterfly Effect | Similitud Coseno: 0.4982
ID de Review: m/mickey_blue_eyes | Título de la Película: Mickey Blue Eyes | Similitud Coseno: 0.4827


In [36]:
# Ejemplo de consulta
consulta = " Películas de Brad Pitt"
resultados = simular_consulta(consulta, tfidf_vectorizer, tfidf_matrix, reviews_df, movies_df)

Consulta:  Películas de Brad Pitt
Tiempo de procesamiento: 1.0910 segundos
Resultados más relevantes:
ID de Review: m/fury_2015 | Título de la Película: Fury | Similitud Coseno: 0.7354
ID de Review: m/moneyball | Título de la Película: Moneyball | Similitud Coseno: 0.6944
ID de Review: m/ad_astra | Título de la Película: Ad Astra | Similitud Coseno: 0.6934


In [38]:
# Ejemplo de consulta
consulta = "Latin American Movies"
resultados = simular_consulta(consulta, tfidf_vectorizer, tfidf_matrix, reviews_df, movies_df)

Consulta: Latin American Movies
Tiempo de procesamiento: 1.2889 segundos
Resultados más relevantes:
ID de Review: m/duck_season_2006 | Título de la Película: Duck Season (Temporada de patos) | Similitud Coseno: 0.5232
ID de Review: m/lower_city | Título de la Película: Lower City (Cidade Baixa) | Similitud Coseno: 0.5020
ID de Review: m/searchers | Título de la Película: The Searchers | Similitud Coseno: 0.4842
ID de Review: m/glengarry_glen_ross | Título de la Película: Glengarry Glen Ross | Similitud Coseno: 0.4226


In [39]:
# Ejemplo de consulta
consulta = "Italian Movies"
resultados = simular_consulta(consulta, tfidf_vectorizer, tfidf_matrix, reviews_df, movies_df)

Consulta: Italian Movies
Tiempo de procesamiento: 1.0715 segundos
Resultados más relevantes:
ID de Review: m/8-12 | Título de la Película: 8 1/2 | Similitud Coseno: 0.6423
ID de Review: m/dormant_beauty | Título de la Película: Dormant Beauty | Similitud Coseno: 0.5744
ID de Review: m/facing_windows | Título de la Película: Facing Windows | Similitud Coseno: 0.5659
ID de Review: m/danger_diabolik | Título de la Película: Danger: Diabolik | Similitud Coseno: 0.4464
ID de Review: m/heaven | Título de la Película: Heaven | Similitud Coseno: 0.4355


In [40]:
# Ejemplo de consulta
consulta = "Cinema Paradiso"
resultados = simular_consulta(consulta, tfidf_vectorizer, tfidf_matrix, reviews_df, movies_df)

Consulta: Cinema Paradiso
Tiempo de procesamiento: 0.9872 segundos
Resultados más relevantes:
ID de Review: m/cinema_paradiso | Título de la Película: Cinema Paradiso (Nuovo Cinema Paradiso) | Similitud Coseno: 0.7025
ID de Review: m/1101561-1101561-malena | Título de la Película: Malena | Similitud Coseno: 0.6001
ID de Review: m/best_worst_movie | Título de la Película: Best Worst Movie | Similitud Coseno: 0.4939


## **Parte 4: Análisis de Resultados**
Analizaremos la consulta de Cinema Paradiso:
* El primer resultado es perfecto: "Cinema Paradiso" tiene una similitud bastante
alta (0.7025), lo que indica que el sistema está identificando correctamente la película relevante para esta consulta.
* Sin embargo, el segundo y tercer resultado no parecen estar relacionados directamente con la consulta de "Cinema Paradiso", sin embargo es importante recalcar que Cinema Paradiso es un filme italiano y el resto de resultados están muy asociados con el director de la primera



Conclusiones Generales:
 * Los resultados que coinciden estrechamente con la consulta tienen una similitud coseno alta, lo que es un buen indicativo de que el sistema funciona correctamente para identificar la relación entre la consulta y los documentos.
 * Es posible que el sistema está evaluando palabras clave genéricas en lugar de un análisis semántico más profundo.